In [95]:
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
psw='proyecto3'

In [96]:
def categ_año():
    # Conectarse a la base de datos usando SQLAlchemy
    engine = psycopg2.connect(
        dbname="p3",
        user="postgres",
        password=psw,  # Asegúrate de definir la variable 'psw' antes de usarla
        host="databasep3.cut51ddih3ok.us-east-1.rds.amazonaws.com",
        port='5432'
    )
    cursor = engine.cursor()

    # Consulta SQL para contar la proporción de puntajes por año
    query = """
    SELECT periodo, 
       punt_global AS "Puntaje Total",  -- Cambiar el nombre de la columna
       COUNT(*) AS cantidad
       FROM dataP3
       WHERE periodo IN (2019, 2022)
       GROUP BY periodo, punt_global;
    """
    
    # Ejecutar la consulta y obtener el resultado como un DataFrame
    df = pd.read_sql_query(query, engine)

    # Mapear los valores de punt_global según la asignación dada
    mapping = {
        'Insuficiente (0 - 99)': 1,
        'Malo (100 - 199)': 2,
        'Regular (200-299)': 3,
        'Suficiente (300-399)': 4,
        'Sobresaliente (400-500)': 5
    }
    
    # Aplicar el reemplazo de manera más robusta
    df['Puntaje Total'] = df['Puntaje Total'].replace({value: key for key, value in mapping.items()})

    # Filtrar solo los años que te interesan
    df = df[df['periodo'].isin([2019, 2022])]

    # Calcular la proporción de cada categoría en relación con la cantidad total por año
    df['proporcion'] = df.groupby('periodo')['cantidad'].transform(lambda x: x / x.sum())

    # Crear la figura de la gráfica de barras
    fig = px.bar(
        df, 
        x='periodo', 
        y='proporcion',  # Usar la proporción en el eje y
        color='Puntaje Total',  # Actualizar el nombre de la columna
        labels={'proporcion': 'Porcentaje', 'periodo': 'Año'},
        title='Porcentaje por categorías de Puntaje Total ICFES antes y después de pandemia en el departamento de Magdalena',
        category_orders={'periodo': [2019, 2022]},  # Ordenar el eje x
        color_discrete_sequence=px.colors.sequential.Greens,  # Escala de colores verde
        color_discrete_map={'Insuficiente (0 - 99)': 'lime',
                            'Malo (100 - 199)': 'palegreen',
                            'Regular (200-299)': 'mediumseagreen',
                            'Suficiente (300-399)': 'seagreen',
                            'Sobresaliente (400-500)': 'darkgreen'},
        text=df['proporcion']  # No aplicar formato aquí
    )   
    
    # Personalizar el diseño de la gráfica
    fig.update_xaxes(title_text='', showticklabels=False)  # Ocultar etiquetas y valores del eje x
    fig.update_yaxes(showgrid=False, showline=False, zeroline=False, title_text='', showticklabels=False)  # Ocultar etiquetas y valores del eje y

    # Personalizar el formato del texto y centrarlo
    fig.update_traces(texttemplate='%{text:.2%}', 
                      textfont=dict(color='white', size=14, family='Arial, sans-serif'), 
                      insidetextanchor='start',
                      textposition='inside')

    # Crear la aplicación Dash
    app = dash.Dash(__name__)

    app.layout = html.Div([
        dcc.Graph(figure=fig)
    ])

    # Ejecutar la aplicación Dash
    if __name__ == '__main__':
        app.run_server(debug=True)

# Asegúrate de tener definida la variable 'psw' antes de llamar a la función
categ_año()




C:\Users\Sebastian\AppData\Local\Temp\ipykernel_2916\1985254552.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

